# Strange issue with models



## Copy of simulate_3d
- For making dataset, can skip this section

In [1]:
%matplotlib inline
import gammapy
gammapy.__version__

'0.17.dev499+gb670519b6'

In [2]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.irf import load_cta_irfs
from gammapy.maps import WcsGeom, MapAxis
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    GaussianSpatialModel,
    SkyModel,
)
from gammapy.makers import MapDatasetMaker, SafeMaskMaker
from gammapy.modeling import Fit
from gammapy.data import Observation
from gammapy.datasets import MapDataset

In [3]:
# Loading IRFs
irfs = load_cta_irfs(
    "$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits"
)

Invalid unit found in background table! Assuming (s-1 MeV-1 sr-1)


In [4]:
# Define the observation parameters (typically the observation duration and the pointing position):
livetime = 2.0 * u.hr
pointing = SkyCoord(0, 0, unit="deg", frame="galactic")

In [5]:
# Define map geometry for binned simulation
energy_reco = MapAxis.from_edges(
    np.logspace(-1.0, 1.0, 10), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(0, 0),
    binsz=0.02,
    width=(6, 6),
    frame="galactic",
    axes=[energy_reco],
)
# It is usually useful to have a separate binning for the true energy axis
energy_true = MapAxis.from_edges(
    np.logspace(-1.5, 1.5, 30), unit="TeV", name="energy", interp="log"
)

In [6]:
# Define sky model to used simulate the data.
# Here we use a Gaussian spatial model and a Power Law spectral model.
spatial_model = GaussianSpatialModel(
    lon_0="0.2 deg", lat_0="0.1 deg", sigma="0.3 deg", frame="galactic"
)
spectral_model = PowerLawSpectralModel(
    index=3, amplitude="1e-11 cm-2 s-1 TeV-1", reference="1 TeV"
)
model_simu = SkyModel(
    spatial_model=spatial_model,
    spectral_model=spectral_model,
    name="model-simu",
)
print(model_simu)

SkyModel

  Name                      : model-simu
  Datasets names            : None
  Spectral model type       : PowerLawSpectralModel
  Spatial  model type       : GaussianSpatialModel
  Temporal model type       : None
  Parameters:
    index                   :   3.000              
    amplitude               :   1.00e-11  1 / (cm2 s TeV)
    reference    (frozen)   :   1.000  TeV         
    lon_0                   :   0.200  deg         
    lat_0                   :   0.100  deg         
    sigma                   :   0.300  deg         
    e            (frozen)   :   0.000              
    phi          (frozen)   :   0.000  deg         




In [7]:
# Create an in-memory observation
obs = Observation.create(pointing=pointing, livetime=livetime, irfs=irfs)
print(obs)

Observation

	obs id            : 0 
 	tstart            : 51544.00
	tstop             : 51544.08
	duration          : 7200.00 s
	pointing (icrs)   : 266.4 deg, -28.9 deg

	deadtime fraction : 0.0%



In [8]:
# Make the MapDataset
empty = MapDataset.create(geom, name="dataset-simu")
maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])
maker_safe_mask = SafeMaskMaker(methods=["offset-max"], offset_max=4.0 * u.deg)
dataset = maker.run(empty, obs)
dataset = maker_safe_mask.run(dataset, obs)
print(dataset)

MapDataset
----------

  Name                            : dataset-simu 

  Total counts                    : nan 
  Total predicted counts          : 161422.07
  Total background counts         : 161422.07

  Exposure min                    : 6.41e+07 m2 s
  Exposure max                    : 2.53e+10 m2 s

  Number of total bins            : 0 
  Number of fit bins              : 804492 

  Fit statistic type              : cash
  Fit statistic value (-2 log(L)) : nan

  Number of models                : 1 
  Number of parameters            : 3
  Number of free parameters       : 1

  Component 0: BackgroundModel
  
    Name                      : dataset-simu-bkg
    Datasets names            : ['dataset-simu']
    Parameters:
      norm                    :   1.000              
      tilt         (frozen)   :   0.000              
      reference    (frozen)   :   1.000  TeV         
  
  


In [9]:
# Add the model on the dataset and Poission fluctuate
dataset.models.append(model_simu)
dataset.fake()
# Do a print on the dataset - there is now a counts maps
print(dataset)

MapDataset
----------

  Name                            : dataset-simu 

  Total counts                    : 169811 
  Total predicted counts          : 170075.03
  Total background counts         : 161422.07

  Exposure min                    : 6.41e+07 m2 s
  Exposure max                    : 2.53e+10 m2 s

  Number of total bins            : 810000 
  Number of fit bins              : 804492 

  Fit statistic type              : cash
  Fit statistic value (-2 log(L)) : 561045.66

  Number of models                : 2 
  Number of parameters            : 11
  Number of free parameters       : 6

  Component 0: BackgroundModel
  
    Name                      : dataset-simu-bkg
    Datasets names            : ['dataset-simu']
    Parameters:
      norm                    :   1.000              
      tilt         (frozen)   :   0.000              
      reference    (frozen)   :   1.000  TeV         
  
  Component 1: SkyModel
  
    Name                      : model-simu
    Dataset

Now use this dataset as you would in all standard analysis. You can plot the maps, or proceed with your custom analysis. 
In the next section, we show the standard 3D fitting as in [analysis_3d](analysis_3d.ipynb).

## The main issue

To reproduce the issue of adding a model, removing it, and putting it back...

In [10]:
# Make a copy of the dataset
dataset_fit = dataset.copy(name="dataset-fit")

First add a sky model to the dataset, named `model_fit`

In [11]:
spatial_model1 = GaussianSpatialModel(
    lon_0="0.1 deg", lat_0="0.1 deg", sigma="0.5 deg", frame="galactic"
)
spectral_model1 = PowerLawSpectralModel(
    index=2, amplitude="1e-9 cm-2 s-1 TeV-1", reference="1 TeV"
)
model_fit = SkyModel(
    spatial_model=spatial_model1,
    spectral_model=spectral_model1,
    name="model-fit",
)

dataset_fit.models = [dataset_fit.models[0], model_fit]
print(dataset_fit.models)

ProperModels

Component 0: BackgroundModel

  Name                      : dataset-fit-bkg
  Datasets names            : ['dataset-fit']
  Parameters:
    norm                    :   1.000              
    tilt         (frozen)   :   0.000              
    reference    (frozen)   :   1.000  TeV         

Component 1: SkyModel

  Name                      : model-fit
  Datasets names            : None
  Spectral model type       : PowerLawSpectralModel
  Spatial  model type       : GaussianSpatialModel
  Temporal model type       : None
  Parameters:
    index                   :   2.000              
    amplitude               :   1.00e-09  1 / (cm2 s TeV)
    reference    (frozen)   :   1.000  TeV         
    lon_0                   :   0.100  deg         
    lat_0                   :   0.100  deg         
    sigma                   :   0.500  deg         
    e            (frozen)   :   0.000              
    phi          (frozen)   :   0.000  deg         




In [12]:
npred1 = dataset_fit.npred().data.sum()
print(npred1)

474656.795929658


For some reason, I do not like this model and remove it. This works well.

In [13]:
dataset_fit.models.pop("model-fit")

SkyModel(spatial_model=<gammapy.modeling.models.spatial.GaussianSpatialModel object at 0x10eade2e8>, spectral_model=<gammapy.modeling.models.spectral.PowerLawSpectralModel object at 0x10eade2b0>)temporal_model=None)

In [14]:
dataset_fit.npred().data.sum()

161422.06647709478

In [15]:
dataset_fit.models.names

['dataset-fit-bkg']

The model has been successfully removed, and the npred computed is okay.
Now, I want to redefine `model-fit` with different parameters and attach it to the dataset.
To make the difference obvious, I just change the amplitude here (it was 1e-9 in the first step, and is 1e-12 now)

In [16]:
spatial_model1 = GaussianSpatialModel(
    lon_0="0.1 deg", lat_0="0.1 deg", sigma="0.5 deg", frame="galactic"
)
spectral_model1 = PowerLawSpectralModel(
    index=2, amplitude="1e-12 cm-2 s-1 TeV-1", reference="1 TeV"
)
model_fit = SkyModel(
    spatial_model=spatial_model1,
    spectral_model=spectral_model1,
    name="model-fit",
)

In [17]:
dataset_fit.models.append(model_fit)

In [18]:
#Look at the npred
dataset_fit.npred().data.sum(), npred1

(474656.795929658, 474656.795929658)

It is the same as npred1 !!! Even though the new amplitude is correctly shown.

In [19]:
dataset_fit.models['model-fit'].parameters.to_table()

name,value,unit,min,max,frozen,error
str9,float64,str14,float64,float64,bool,int64
index,2.000e+00,,nan,nan,False,0.000e+00
amplitude,1.000e-12,cm-2 s-1 TeV-1,nan,nan,False,0.000e+00
reference,1.000e+00,TeV,nan,nan,True,0.000e+00
lon_0,1.000e-01,deg,nan,nan,False,0.000e+00
lat_0,1.000e-01,deg,-9.000e+01,9.000e+01,False,0.000e+00
sigma,5.000e-01,deg,0.000e+00,nan,False,0.000e+00
e,0.000e+00,,0.000e+00,1.000e+00,True,0.000e+00
phi,0.000e+00,deg,nan,nan,True,0.000e+00
